In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

In [2]:
from flask import Flask, render_template, request
from PIL import Image
from io import BytesIO
import torch
from torchvision.transforms import ToTensor

### Initialize Model

In [3]:
class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Input 1x28x28, Output 32x28x28
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output 32x14x14

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Input 32x14x14, Output 64x14x14
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output 64x7x7

        # Fully Connected Layers
        self.fc1 = nn.Linear(64 * 7 * 7, 512)  # Input 64*7*7=3136, Output 512
        self.act3 = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)  # Input 512, Output 10 (number of classes)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = self.act3(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
training_data = datasets.MNIST(root=".", train=True, download=True, transform=ToTensor())
# training_data = datasets.FashionMNIST(root=".", train=True, download=True, transform=ToTensor())

test_data = datasets.MNIST(root=".", train=False, download=True, transform=ToTensor())
# test_data = datasets.FashionMNIST(root=".", train=False, download=True, transform=ToTensor())

### Load Trained Model from Google Colab (5 Epochs, 90% accuracy, T4 GPU)

In [19]:
# Load Trained Model

device = torch.device('cpu')
model = MNISTModel()

PATH = 'trained_model_acc_90.pth'
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

### Predict Image Locally

In [23]:
from PIL import Image
from torchvision.transforms import ToTensor, Normalize
import torch


# Define the normalization transform (if used during training)
normalize = Normalize(mean=[0.5], std=[0.5])  # Example normalization for images with pixel values in [0, 1]

# Preprocess the test image
test_image_path = 'test/paint-5.png'
test_image = Image.open(test_image_path).convert('L')  # Convert to grayscale if necessary
test_image = test_image.resize((28, 28))  # Resize to match model input size
test_image_tensor = ToTensor()(test_image).unsqueeze(0)  # Convert to tensor and add batch dimension

# Apply normalization if used during training
# test_image_tensor = normalize(test_image_tensor)

# Perform prediction
with torch.no_grad():
    output = model(test_image_tensor)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

# Display prediction result
print(f"Predicted class: {predicted_class}")
print(f"Output probabilities: {probabilities.squeeze().tolist()}")


Predicted class: 5
Output probabilities: [0.00010912139987340197, 1.492532106794897e-07, 0.011908882297575474, 0.06017802283167839, 1.2147648220661722e-08, 0.927375853061676, 2.7306637662150024e-07, 2.6558485842542723e-05, 0.00040040345629677176, 8.415304932896106e-07]


### Predict Image in Flask Web-App

In [24]:
from flask import Flask, render_template, request
from PIL import Image
from torchvision.transforms import ToTensor, Normalize
from io import BytesIO
import torch

app = Flask(__name__, template_folder='templates', static_url_path='/static')

# Define the normalization transform
normalize = Normalize(mean=[0.5], std=[0.5])  # Assuming input pixel range [0, 255]

def initialize_flask_app():
    # Define route (endpoint) for home page
    @app.route('/', methods=['GET'])
    def home():
        return render_template('index.html')

    # Define route (endpoint) for handling image upload and prediction
    @app.route('/predict', methods=['POST'])
    def predict():
        if 'image' in request.files:
            # Read and preprocess the uploaded image
            img = Image.open(BytesIO(request.files['image'].read()))
            img = img.convert('L')  # Convert to grayscale
            img = img.resize((28, 28))  # Resize image to 28x28
            img = ToTensor()(img).unsqueeze(0)  # Convert to tensor and add batch dimension
            img = normalize(img)  # Apply normalization

            with torch.no_grad():
                output = model(img)
                _, predicted = torch.max(output, 1)
                prediction = predicted.item()

            return render_template('result.html', prediction=prediction)
        else:
            return 'Error: No image provided.'

    # Run the Flask app
    if __name__ == '__main__':
        app.run()

if __name__ == '__main__':
    initialize_flask_app()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/May/2024 14:40:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2024 14:40:44] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2024 14:40:44] "GET /static/index.js HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2024 14:40:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/May/2024 14:40:51] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2024 14:40:51] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2024 14:40:58] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2024 14:40:58] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2024 14:41:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2024 14:41:04] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2024 14:45:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2024 14:45:07] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [13/May/2024 14:45:17] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - 